# Hash 对字符串s实现16/32位加密

In [1]:
import hashlib
def hash_cry32(s):
    m = hashlib.md5()
    m.update((str(s).encode('utf-8')))
    return m.hexdigest()
print(hash_cry32(1)) 
print(hash_cry32('hello'))

c4ca4238a0b923820dcc509a6f75849b
5d41402abc4b2a76b9719d911017c592


<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/26D82872-AD05-4B65-B5E4-0D7B1AD331A7.png">

In [2]:
import hashlib
hash  = hashlib.md5()

hash.update(('Hello, ').encode('utf-8'))
ret2 = hash.hexdigest()
print({"code":"hash.hexdigest","类型":type(ret2), "长度":len(ret2), "密文":ret2})

{'code': 'hash.hexdigest', '类型': <class 'str'>, '长度': 32, '密文': 'c84cabbaebee9a9631c8be234ac64c26'}


In [3]:
hash.update(('World!').encode('utf-8'))
ret1 = hash.digest()
print({"code":"hash.digest()","类型":type(ret1), "长度":len(ret1), "密文":ret1})

{'code': 'hash.digest()', '类型': <class 'bytes'>, '长度': 16, '密文': b'e\xa8\xe2}\x88y(81\xb6d\xbd\x8b\x7f\n\xd4'}


# 五、base64模块介绍
#### 经常听到有人说“base64加密”，其实base64并不能用于数据加密，它也不是为了纯粹的数据加密而生的，它的出现是为了解决不可见字符串的网络传输和数据保存问题。因为，用base64对数据进行转换的过程不能成为“加密”与“解密”，只能成为“编码”与“解码”。下面我们也会用到它，所以这里顺便做下简单的介绍。

### 1. base64的作用
#### Base64是一种用64个字符来表示任意二进制数据的方法，它是一种通过查表对二进制数据进行编码的方法，不能用于数据加密。base64最初的出现时为了能够正确的传输邮件数据，因为邮件中的附件（比如图片）的二进制数中可能存在不可见字符（ascii码中128-255之间的值是不可见字符），比如我们尝试用记事本或其他文本编辑器打开一个图片时，通常都会看到一大堆乱码，这些乱码就是不可见字符。由于早期的一些网络设备和网络协议是无法正确识别这些字符的，这就可能在数据传输时出现各种无法预知的问题。base64的作用就是把含有不可见字符的信息用可见字符来表示（Ascii码中0-127之间的值是可见字符），从而解决这个问题。

#### 关于base64的介绍及实现原理可以看看这几篇文章：

#### http://www.cnblogs.com/wellsoho/archive/2009/12/09/1619924.html
https://www.zhihu.com/question/36306744/answer/
http://www.liaoxuefeng.com/wiki/001374738125095c955c1e6d8bb493182103fac9270762a000/001399413803339f4bbda5c01fc479cbea98b1387390748000
#### 2. base64的常见应用场景
#### base64适用于小段内容的编码，比如数字证书的签名、URL、Cookie信息等需要通过网络进行传输的小段数据。关于base64在数字签名中的应用会在本文后面讲解pycrypto模块使用实例时有具体的应用示例。

### 3. base64模块介绍及简单使用示例
#### Python中有一个内置的base64模块可直接用来进行base64的编码和解码工作--即提供 “二进制数据” 与 “可打印（可见）的ASCII字符”之间的转换功能。常用的函数有以下几个：
<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/C9124FAC-7E51-440F-9B28-06C7C01B4C95.png">

In [4]:
import base64

data = ('hello, 世界！').encode('utf-8')
based_data1 = base64.b64encode(data)
plain_data1 = base64.b64decode(based_data1)
based_data2 = base64.urlsafe_b64encode(data)
plain_data2 = base64.urlsafe_b64decode(based_data2)
print(based_data1)
print(based_data2)
print(plain_data1.decode('utf-8'))
print(plain_data2.decode('utf-8'))

b'aGVsbG8sIOS4lueVjO+8gQ=='
b'aGVsbG8sIOS4lueVjO-8gQ=='
hello, 世界！
hello, 世界！


#  3. base64编码结果后的等号'='
#### 通过上面的这个简单示例的输出结果会发现，随翻urlsafe_b64encode()函数会把编码结果中的字符'+'和字符'/'替换成其他URL的非保留字符，但是它的编码结果中还是可能出现字符'='。那么这些字符'='代表什么呢？能否去掉呢？

#### 其实base64编码的过程中会先把原来数据中的每3个字节的二进制数据编码为4个字节的文本数据，当原始数据最后不满3个字节时就需要用'\00'字节进行补位凑够3个字节，而且会在编码结果的最后加上相应个数的'='号来表示补了多少个字节，这样解码的时候就可以去掉那些补位的字节。

### 由此我们可以得出两个结论：

## $$ base64编码后的结果的末尾可能存在字符'='个数分别是：0个、1个和2 $$
## $$ base64编码后的结果应该是4的倍数。$$
#  基于以上第2个结论，为了避免编码结果中可能出现的的字符'='对网络数据传输造成影响，可以在传出前去掉后面的字符'='，接收方可以通过对数据的长度对4求模得到应该补上的字符'='个数，从而得到正确的数据。比如，我们可以通过下面这个解码函数来完成这个过程：

In [5]:
import base64

def safe_b64decode(s):
    length = len(s) % 4
    for i in range(length):
        s = s + '='
    return base64.b64decode(s)

if __name__ == '__main__':
    print(safe_b64decode('aGVsbG8sIOS4lueVjO+8gQ==').decode('utf-8'))
    print(safe_b64decode('aGVsbG8sIOS4lueVjO+8gQ=').decode('utf-8'))
    print(safe_b64decode('aGVsbG8sIOS4lueVjO+8gQ').decode('utf-8'))

hello, 世界！
hello, 世界！
hello, 世界！


<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/FDC720F7-6B03-4748-B9A0-5BFA1C8D5D3E.png">

In [6]:
from Crypto.Hash import MD5

## hash SHA256 加密

In [7]:
from Crypto.Hash import SHA256

hash = SHA256.new()
hash.update('Hello, World!'.encode('utf-8'))
digest = hash.hexdigest()
print(digest)

dffd6021bb2bd5b0af676290809ec3a53191dd81c7f70a4b28688a362182986f


## AES加密 与 解密

### 设置参数

In [8]:
from Crypto.Cipher import AES

# 加密与解密所使用的密钥，长度必须是16的倍数
secret_key = "ThisIs SecretKey" 
# 要加密的明文数据，长度必须是16的倍数
plain_data = "Hello, World123!"
# IV参数，长度必须是16的倍数
iv_param = 'This is an IV456'


#### AES数据加密

In [9]:
aes1 = AES.new(secret_key, AES.MODE_CBC, iv_param)
cipher_data = aes1.encrypt(plain_data)
print('cipher data：', cipher_data)

cipher data： b'\xcb\x7fd\x03\x12T,\xbe\x91\xac\x1a\xd5\xaa\xe6P\x9a'


#### AES数据解密

In [10]:
aes2 = AES.new(secret_key, AES.MODE_CBC, 'This is an IV456')
plain_data2 = aes2.decrypt(cipher_data)  # 解密后的明文数据
print('plain text：', plain_data2.decode('utf-8'))

plain text： Hello, World123!


### 随机数操作

In [11]:
from Crypto.Random import random

print('random.randint: ', random.randint(10, 20))
print('random.randrange: ', random.randrange(10, 20, 2))
print('random.randint: ', random.getrandbits(3))
print('random.choice: ', random.choice([1, 2, 3, 4, 5]))
print('random.sample: ', random.sample([1, 2, 3, 4, 5], 3))
list = [1, 2, 3, 4, 5]
random.shuffle(list)
print('random.shuffle: ', list)

random.randint:  19
random.randrange:  16
random.randint:  4
random.choice:  1
random.sample:  [2, 4, 5]
random.shuffle:  [2, 4, 1, 3, 5]


## RSA算法生成密钥对儿

In [12]:
#-*- coding:utf-8 -*-
from Crypto import Random
from Crypto.Hash import SHA
from Crypto.Cipher import PKCS1_v1_5 as Cipher_pkcs1_v1_5
from Crypto.Signature import PKCS1_v1_5 as Signature_pkcs1_v1_5
from Crypto.PublicKey import RSA
import base64


# 加密解密：公钥加密，私钥解密
#
# 签名验签：私钥签名，公钥验签
#
print ("1、生成私钥和公钥")

# 伪随机数生成器
random_generator = Random.new().read
# rsa算法生成实例
rsa = RSA.generate(1024, random_generator)
# A的秘钥对的生成
private_pem = rsa.exportKey()

with open('aaa-private.pem', 'wb') as f:
    f.write(private_pem)

public_pem = rsa.publickey().exportKey()
with open('aaa-public.pem', 'wb') as f:
    f.write(public_pem)

# B的秘钥对的生成
private_pem = rsa.exportKey()
with open('bbb-private.pem', 'wb') as f:
    f.write(private_pem)

public_pem = rsa.publickey().exportKey()
with open('bbb-public.pem', 'wb') as f:
    f.write(public_pem)

# 加密和解密
print ("2、加密和解密")
# A使用B的公钥对内容进行rsa 加密

message = '大家好，这就是我要加密的数据'
print ("message: " + message)
with open('bbb-public.pem') as f:
    key = f.read()
    rsakey = RSA.importKey(str(key))
    cipher = Cipher_pkcs1_v1_5.new(rsakey)
    cipher_text = base64.b64encode(cipher.encrypt(bytes(message.encode("utf8"))))
    print ("加密（encrypt）")
    print (cipher_text)

# B使用自己的私钥对内容进行rsa 解密

with open('bbb-private.pem') as f:
    key = f.read()
    rsakey = RSA.importKey(key)
    cipher = Cipher_pkcs1_v1_5.new(rsakey)
    text = cipher.decrypt(base64.b64decode(cipher_text), random_generator)
    print( "解密（decrypt）")
    print ("text:" + str(text,"utf8"))
    print("message:"+message)

    assert str(text,"utf8") == message

# 签名与验签
print ("3、 签名与验签")

# A使用自己的私钥对内容进行签名
print( "签名")
with open('aaa-private.pem') as f:
    key = f.read()
    rsakey = RSA.importKey(key)
    signer = Signature_pkcs1_v1_5.new(rsakey)
    digest = SHA.new()
    digest.update(message.encode("utf8"))
    sign = signer.sign(digest)
    signature = base64.b64encode(sign)

print(signature)
#B使用A的公钥进行验签
print ("验签")
with open('aaa-public.pem') as f:
    key = f.read()
    rsakey = RSA.importKey(key)
    verifier = Signature_pkcs1_v1_5.new(rsakey)
    digest = SHA.new()
    digest.update(message.encode("utf8"))
    is_verify = verifier.verify(digest, base64.b64decode(signature))
print (is_verify)

1、生成私钥和公钥
2、加密和解密
message: 大家好，这就是我要加密的数据
加密（encrypt）
b'kZJ52nspV9xAl/v9RXIwd5AgsnFwE4EcTOlVZLmxgjCFf6A6muFLJC8pFTQddXi8pB3KAIFSAy0lFdb8h4Sz8dg9EvK+M0TLJw0qMCKfwSowWEvzWYhOCtvfHIIE5/ZqRpay7mxlxxuea8Nl/ePVSGvBTx9sLFBk35WhShM/5mU='
解密（decrypt）
text:大家好，这就是我要加密的数据
message:大家好，这就是我要加密的数据
3、 签名与验签
签名
b'D7jewCAiZU7J4lRcfkuIQaZJgI/jsLg/AJkZX/ipiXGv7dlV11XIXU6YbCFjHG9boyBj14TfxwOQ6wT/BtUuFGtUjtSIBfFd59peVyDRn67osbxOiMp4whjCBb/OysFNh5rCENDOZwsqP+ewV/+pfhfcqAd1lwu24aWvGU4JV8E='
验签
True


参考资料 https://www.cnblogs.com/yyds/p/7072492.html

官方文档 https://www.dlitz.net/software/pycrypto/

简书 https://www.jianshu.com/p/9e9693d70aba